# if u like it up vote it😁

# Importing libraries

In [ ]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS
from datetime import datetime
from PIL import Image

**Counrting total no. of files and folders :**

In [ ]:
total_folders = total_files = 0
folder_info = []
images_names = []
for base, dirs, files in tqdm(os.walk('/kaggle/input/h-and-m-personalized-fashion-recommendations/')):
    for directories in dirs:
        folder_info.append((directories, len(os.listdir(os.path.join(base, directories)))))
        total_folders += 1
    for _files in files:
        total_files += 1
        if len(_files.split(".jpg"))==2:
            images_names.append(_files.split(".jpg")[0])

In [ ]:
print(f"Total number of folders: {total_folders}\nTotal number of files: {total_files}")
folder_info_df = pd.DataFrame(folder_info, columns=["folder", "files count"])
folder_info_df.sort_values(["files count"], ascending=False).head()

# Reading the data 

In [ ]:
articles_df = pd.read_csv("/kaggle/input/h-and-m-personalized-fashion-recommendations/articles.csv")
customers_df = pd.read_csv("/kaggle/input/h-and-m-personalized-fashion-recommendations/customers.csv")
sample_submission_df = pd.read_csv("/kaggle/input/h-and-m-personalized-fashion-recommendations/sample_submission.csv")
transactions_train_df = pd.read_csv("/kaggle/input/h-and-m-personalized-fashion-recommendations/transactions_train.csv")

In [ ]:
articles_df

In [ ]:
customers_df

In [ ]:
transactions_train_df

In [ ]:
sample_submission_df

# About data 


* There are 4 tabels :
 1. articles - contains informations about each article (105542 rows × 25 columns)
 2. customers - contains informations about each customer (1371980 rows × 7 columns)
 3. transactions (31788324 rows × 5 columns)
 4. sample submission(1371980 rows × 2 columns)
 
* Foreign keys for the customer and articles tables= 'customer_id` & 'article_id'
 
* Transaction train data has entries for the date of the transaction, the customer id, the article id, a price (per transaction) and a sales channel id.

In [ ]:
# Function to count missing values
def missing_data(data):
    total = data.isnull().sum().sort_values(ascending = False)
    percent = (data.isnull().sum()/data.isnull().count()*100).sort_values(ascending = False)
    return pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])

In [ ]:
def unique_values(data):
    total = data.count()
    tt = pd.DataFrame(total)
    tt.columns = ['Total']
    uniques = []
    for col in data.columns:
        unique = data[col].nunique()
        uniques.append(unique)
    tt['Uniques'] = uniques
    return tt

In [ ]:
articles_df.info()

In [ ]:
missing_data(articles_df)

In articles_df only details_desc has 416 missing values that is 0.394156 %.

In [ ]:
customers_df.info()

In [ ]:
missing_data(customers_df)

Active and FN has more than 50% of missing rest has a minor amount of missing data

In [ ]:
sample_submission_df.info()

In [ ]:
transactions_train_df.info()

# Data visualization

In [ ]:

temp = articles_df.groupby(["product_group_name"])["product_type_name"].nunique()
df = pd.DataFrame({'Product Group': temp.index,
                   'Product Types': temp.values
                  })

df = df.sort_values(['Product Types'], ascending=False)
fig, ax = plt.subplots( figsize=(10,8))

ax.tick_params(axis='x', which='major', labelsize=15)
ax.tick_params(axis='y', which='major', labelsize=15)
ax.set_ylabel('Product group', weight='semibold', fontsize = 20)
ax.set_xlabel('Prodiuct type', weight='semibold', fontsize = 20)

plt.title('Number of Product Types per each Product Group', weight='semibold', fontsize = 20)
sns.set_color_codes("pastel")
s = sns.barplot(x = 'Product Group', y="Product Types", data=df)

ax.grid(b = True, color ='grey',
        linestyle ='-.', linewidth = 0.5,
        alpha = 0.6)
s.set_xticklabels(s.get_xticklabels(),rotation=90)
locs, labels = plt.xticks()
plt.show()

In [ ]:
stopwords = set(STOPWORDS)

def show_wordcloud(data, title = None):
    wordcloud = WordCloud(
        background_color='white',
        stopwords=stopwords,
        max_words=200,
        max_font_size=40, 
        scale=5,
        random_state=1
    ).generate(str(data))

    fig = plt.figure(1, figsize=(10,10))
    plt.axis('off')
    if title: 
        fig.suptitle(title, fontsize=14)
        fig.subplots_adjust(top=2.3)

    plt.imshow(wordcloud)
    plt.show()

In [ ]:
show_wordcloud(articles_df["prod_name"], "Wordcloud from product name")

In [ ]:

sns.set_theme(style="darkgrid")
temp = articles_df.groupby(["product_group_name"])["product_type_name"].nunique()
df = pd.DataFrame({'Product Group': temp.index,
                   'Product Types': temp.values
                  })

df = df.sort_values(['Product Types'], ascending=False)

f, ax = plt.subplots(figsize=(10, 16))


sns.set_color_codes("pastel")
sns.barplot(x=df['Product Types'],
                y=df['Product Group'],
                linewidth=0.5, saturation = 0.5)


    
plt.title('Number of Product Types per each Product Group', weight='bold',fontsize=15)
plt.xlabel("Product type", weight='semibold',fontsize=15)
plt.ylabel("Product Group", weight='semibold',fontsize=15)


In [ ]:

temp = articles_df.groupby(["product_type_name"])["article_id"].nunique()
df = pd.DataFrame({'Product Type': temp.index,
                   'Articles': temp.values
                  })
total_types = len(df['Product Type'].unique())
df = df.sort_values(['Articles'], ascending=False)[0:50]

f, ax = plt.subplots(figsize=(10, 16))


sns.set_color_codes("pastel")
sns.barplot(x=df['Articles'],
                y=df['Product Type'],
                linewidth=0.5, saturation = 0.5)


    
plt.title(f'Number of Articles per each Product Type (top 50 from total: {total_types})', weight='bold',fontsize=15)
plt.xlabel("Articles", weight='semibold',fontsize=15)
plt.ylabel("Product Type", weight='semibold',fontsize=15)


In [ ]:

temp = articles_df.groupby(["department_name"])["article_id"].nunique()
df = pd.DataFrame({'Department Name': temp.index,
                   'Articles': temp.values
                  })
total_depts = len(df['Department Name'].unique())
df = df.sort_values(['Articles'], ascending=False).head(50)

f, ax = plt.subplots(figsize=(10, 16))


sns.set_color_codes("pastel")
sns.barplot(x=df['Articles'],
                y=df['Department Name'],
                linewidth=0.5, saturation = 0.5)


    
plt.title(f'Number of Articles per each Department (top 50 from total: {total_depts})', weight='bold',fontsize=15)
plt.xlabel("Articles", weight='semibold',fontsize=15)
plt.ylabel("Department Name", weight='semibold',fontsize=15)


In [ ]:
temp = articles_df.groupby(["index_group_name"])["article_id"].nunique()
df = pd.DataFrame({'Index Group Name': temp.index,
                   'Articles': temp.values
                  })
df = df.sort_values(['Articles'], ascending=False)
plt.figure(figsize = (6,6))
plt.title(f'Number of Articles per each Index Group Name')
sns.set_color_codes("pastel")
s = sns.barplot(x = 'Index Group Name', y="Articles", data=df)
s.set_xticklabels(s.get_xticklabels(),rotation=90)
locs, labels = plt.xticks()
plt.show()

In [ ]:

temp = articles_df.groupby(["colour_group_name"])["article_id"].nunique()
df = pd.DataFrame({'Colour Group Name': temp.index,
                   'Articles': temp.values
                  })
df = df.sort_values(['Articles'], ascending=False)

f, ax = plt.subplots(figsize=(10, 16))


sns.set_color_codes("pastel")
sns.barplot(x=df['Articles'],
                y=df['Colour Group Name'],
                linewidth=0.5, saturation = 0.5)


    
plt.title(f'Number of Articles per each Colour Group Name', weight='bold',fontsize=15)
plt.xlabel("Articles", weight='semibold',fontsize=15)
plt.ylabel('Colour Group Name', weight='semibold',fontsize=15)


In [ ]:
temp = articles_df.groupby(["perceived_colour_value_name"])["article_id"].nunique()
df = pd.DataFrame({'Perceived Colour Group Name': temp.index,
                   'Articles': temp.values
                  })
df = df.sort_values(['Articles'], ascending=False)
plt.figure(figsize = (6,6))
plt.title(f'Number of Articles per each Perceived Colour Group Name')
sns.set_color_codes("pastel")
s = sns.barplot(x = 'Perceived Colour Group Name', y="Articles", data=df)
s.set_xticklabels(s.get_xticklabels(),rotation=90)
locs, labels = plt.xticks()
plt.show()

In [ ]:
temp = articles_df.groupby(["perceived_colour_master_name"])["article_id"].nunique()
df = pd.DataFrame({'Perceived Colour Master Name': temp.index,
                   'Articles': temp.values
                  })
df = df.sort_values(['Articles'], ascending=False)
plt.figure(figsize = (12,6))
plt.title(f'Number of Articles per each Perceived Colour Master Name')
sns.set_color_codes("pastel")
s = sns.barplot(x = 'Perceived Colour Master Name', y="Articles", data=df)
s.set_xticklabels(s.get_xticklabels(),rotation=90)
locs, labels = plt.xticks()
plt.show()

In [ ]:
temp = articles_df.groupby(["index_name"])["article_id"].nunique()
df = pd.DataFrame({'Index Name': temp.index,
                   'Articles': temp.values
                  })
df = df.sort_values(['Articles'], ascending=False)
plt.figure(figsize=(15,10))
explode_values = (0, 0, 0.2, 0, 0, 0, 0.2, 0, 0, 0, 0, 0.2,0.2)
    
circle = plt.Circle((0,0), 0.7, color='white')
plt.rcParams['text.color'] = 'black'
    
plt.pie(df['Articles'],
        labels=df['Index Name'],
        shadow=True,autopct='%.1f%%' )
    
p = plt.gcf()
p.gca().add_artist(circle)
plt.title(f'Number of Articles per each Index Name', size=20)
plt.show()

In [ ]:
temp = articles_df.groupby(["garment_group_name"])["article_id"].nunique()
df = pd.DataFrame({'Garment Group Name': temp.index,
                   'Articles': temp.values
                  })
df = df.sort_values(['Articles'], ascending=False)
plt.figure(figsize=(15,10))
explode_values = (0, 0, 0.2, 0, 0, 0, 0.2, 0, 0, 0, 0, 0.2,0.2)
    
circle = plt.Circle((0,0), 0.7, color='white')
plt.rcParams['text.color'] = 'black'
    
plt.pie(df['Articles'],
        labels=df['Garment Group Name'],
        shadow=True,autopct='%.1f%%' )
    
p = plt.gcf()
p.gca().add_artist(circle)
plt.title(f'Number of Articles per each Garment Group Name', size=20)
plt.show()

# Customers data

In [ ]:
temp = customers_df.groupby(["age"])["customer_id"].count()
df = pd.DataFrame({'Age': temp.index,
                   'Customers': temp.values
                  })
df = df.sort_values(['Age'], ascending=False)
plt.figure(figsize = (16,6))
plt.title(f'Number of Customers per each Age')
sns.set_color_codes("pastel")
s = sns.barplot(x = 'Age', y="Customers", data=df)
s.set_xticklabels(s.get_xticklabels(),rotation=90)
locs, labels = plt.xticks()
plt.show()

In [ ]:
temp = customers_df.groupby(["fashion_news_frequency"])["customer_id"].count()
df = pd.DataFrame({'Fashion News Frequency': temp.index,
                   'Customers': temp.values
                  })
df = df.sort_values(['Customers'], ascending=False)
plt.figure(figsize=(15,10))
explode_values = (0, 0, 0.2, 0, 0, 0, 0.2, 0, 0, 0, 0, 0.2,0.2)
    
circle = plt.Circle((0,0), 0.7, color='white')
plt.rcParams['text.color'] = 'black'
    
plt.pie(df['Customers'],
        labels=df['Fashion News Frequency'],
        shadow=True,autopct='%.1f%%' )
    
p = plt.gcf()
p.gca().add_artist(circle)
plt.title(f'Number of Articles per each Garment Group Name', size=20)
plt.show()

In [ ]:
temp = customers_df.groupby(["club_member_status"])["customer_id"].count()
df = pd.DataFrame({'Club Member Status': temp.index,
                   'Customers': temp.values
                  })
df = df.sort_values(['Customers'], ascending=False)
plt.figure(figsize = (6,6))
plt.title(f'Number of Customers per each Club Member Status')
sns.set_color_codes("pastel")
s = sns.barplot(x = 'Club Member Status', y="Customers", data=df)
s.set_xticklabels(s.get_xticklabels(),rotation=90)
locs, labels = plt.xticks()
plt.show()

# Transactions data

In [ ]:
df = transactions_train_df.sample(100_000)
fig, ax = plt.subplots(1, 1, figsize=(14, 7))
sns.kdeplot(np.log(df.loc[df["sales_channel_id"]==1].price.value_counts()))
sns.kdeplot(np.log(df.loc[df["sales_channel_id"]==2].price.value_counts()))
ax.legend(labels=['Sales channel 1', 'Sales channel 1'])
plt.title("Logaritmic distribution of price frequency in transactions, grouped per sales channel (100k sample)")
plt.show()

In [ ]:

df = transactions_train_df.sample(100_000).groupby(["t_dat", "sales_channel_id"])["article_id"].count().reset_index()
df["t_dat"] = df["t_dat"].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
df.columns = ["Date", "Sales Channel Id", "Transactions"]
fig, ax = plt.subplots(1, 1, figsize=(16,6))
g1 = ax.plot(df.loc[df["Sales Channel Id"]==1, "Date"], df.loc[df["Sales Channel Id"]==1, "Transactions"], label="Sales Channel 1", color="Darkblue")
g2 = ax.plot(df.loc[df["Sales Channel Id"]==2, "Date"], df.loc[df["Sales Channel Id"]==2, "Transactions"], label="Sales Channel 2", color="Magenta")
plt.xlabel("Date")
plt.ylabel("Transactions")
ax.legend()
plt.title(f"Transactions per day, grouped by Sales Channel (100k sample; to get the real volume, please consider that real transaction count is {round(transactions_train_df.shape[0]/10.e6,2)}M)")
plt.show()

In [ ]:
df = transactions_train_df.groupby(["t_dat", "sales_channel_id"])["article_id"].nunique().reset_index()
df["t_dat"] = df["t_dat"].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
df.columns = ["Date", "Sales Channel Id", "Unique Articles"]
fig, ax = plt.subplots(1, 1, figsize=(16,6))
g1 = ax.plot(df.loc[df["Sales Channel Id"]==1, "Date"], df.loc[df["Sales Channel Id"]==1, "Unique Articles"], label="Sales Channel 1", color="Blue")
g2 = ax.plot(df.loc[df["Sales Channel Id"]==2, "Date"], df.loc[df["Sales Channel Id"]==2, "Unique Articles"], label="Sales Channel 2", color="Green")
plt.xlabel("Date")
plt.ylabel("Unique Articles / Day")
ax.legend()
plt.title(f"Unique articles per day, grouped by Sales Channel")
plt.show()

In [ ]:
df = transactions_train_df.sample(100_000).groupby(["t_dat"])["article_id"].count().reset_index()
df["t_dat"] = df["t_dat"].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
df.columns = ["Date", "Transactions"]
fig, ax = plt.subplots(1, 1, figsize=(16,6))
plt.plot(df["Date"], df["Transactions"], color="Darkgreen")
plt.xlabel("Date")
plt.ylabel("Transactions")
plt.title(f"Transactions per day (100k sample; to get the real volume, please consider that real transaction count is {round(transactions_train_df.shape[0]/10.e6,2)}M)")
plt.show()


To be continued.......